# Grid Search vs Random Search
From [Scikit-learn documentation](https://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html)

This example evaluates the cross-validated grid search and the cross-validated random search of the parameter space for a small classifier.

In [3]:
!pip install scikit-optimize

     |████████████████████████████████| 100 kB 2.7 MB/s 


In [4]:
import numpy as np

from time import time
import scipy.stats as stats
from sklearn.utils.fixes import loguniform

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.linear_model import SGDClassifier

from skopt import BayesSearchCV

## Load Data & Define Classifier

Linear [Stochastic Gradient Descent Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html) from scikit-learn with a [Hinge loss](https://en.wikipedia.org/wiki/Hinge_loss) for maximum-margin classifiers and [elastic net regularization](https://en.wikipedia.org/wiki/Elastic_net_regularization).

In [5]:
# get some data
X, y = load_digits(return_X_y=True, n_class=3)

# build a classifier
clf = SGDClassifier(loss="hinge", penalty="elasticnet", fit_intercept=True)

## Report Utility

In [6]:
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results["rank_test_score"] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print(
                "Mean validation score: {0:.3f} (std: {1:.3f})".format(
                    results["mean_test_score"][candidate],
                    results["std_test_score"][candidate],
                )
            )
            print("Parameters: {0}".format(results["params"][candidate]))
            print("")

## What we optimize

Define the parameter distribution for `average`, `l1_ratio`, and `alpha` in the SGDCLassifier. 

- `average`: Whether to comput average weights over computations
- `l1_ratio`: ElasticNet mixing parameter, balancing L1 norm and L2 norm regularization.
- `alpha`: Strength of regularization

We set `n_iter` to `15` for a relatively small budget.

## Randomized Search

- Opportunistic search
- Budget independent of No. parameters
- Adding Parameters not Inefficient
- Search continues after minimum
- Possible to miss optimal parameters due to random sampling without direction

In [7]:
# specify parameters and distributions to sample from
param_dist = {
    "average": [True, False],
    "l1_ratio": stats.uniform(0, 1),
    "alpha": loguniform(1e-2, 1e0),
}

# run randomized search
n_iter_search = 15
random_search = RandomizedSearchCV(
    clf, param_distributions=param_dist, n_iter=n_iter_search,
    random_state=42
)

In [8]:
start = time()
random_search.fit(X, y)
print(
    "RandomizedSearchCV took %.2f seconds for %d candidates parameter settings."
    % ((time() - start), n_iter_search)
)
report(random_search.cv_results_)

RandomizedSearchCV took 0.80 seconds for 15 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.989 (std: 0.011)
Parameters: {'alpha': 0.16738085788752124, 'average': False, 'l1_ratio': 0.04666566321361543}

Model with rank: 2
Mean validation score: 0.981 (std: 0.020)
Parameters: {'alpha': 0.010994335574766197, 'average': False, 'l1_ratio': 0.7219987722668247}

Model with rank: 3
Mean validation score: 0.980 (std: 0.016)
Parameters: {'alpha': 0.023270677083837798, 'average': False, 'l1_ratio': 0.6116531604882809}



## Grid Search

This discretizes the search space from the boundaries used in the random search to obtain a grid to search.

- Exhaustive Search
- Every Combination is Evaluated
- Combinatoric Explosion of Evals
- Iterating beyond global minimum to evaluate all gridpoints
- Possible to miss optimal parameters because explicit values are provided

In [9]:
# use a full grid over all parameters
param_grid = {
    "average": [True, False],
    "l1_ratio": np.linspace(0, 1, num=10),
    "alpha": np.power(10, np.arange(-2, 1, dtype=float)),
}

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid)
start = time()
grid_search.fit(X, y)

print(
    "GridSearchCV took %.2f seconds for %d candidate parameter settings."
    % (time() - start, len(grid_search.cv_results_["params"]))
)
report(grid_search.cv_results_)

GridSearchCV took 5.03 seconds for 60 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.994 (std: 0.007)
Parameters: {'alpha': 0.1, 'average': False, 'l1_ratio': 0.2222222222222222}

Model with rank: 2
Mean validation score: 0.993 (std: 0.007)
Parameters: {'alpha': 0.01, 'average': False, 'l1_ratio': 0.1111111111111111}

Model with rank: 2
Mean validation score: 0.993 (std: 0.007)
Parameters: {'alpha': 0.01, 'average': False, 'l1_ratio': 0.2222222222222222}

Model with rank: 2
Mean validation score: 0.993 (std: 0.004)
Parameters: {'alpha': 0.1, 'average': False, 'l1_ratio': 0.6666666666666666}



## Bayesian Optimization

This imports the `scikit-optimize` package to compare a Bayesian Search for optimal parameters. 

This uses the same boundaries as the random search and `15` iterations. 

- Search based on former parameters
- Bayesian optimization
- Converges to a minimum
- Adding parameters adds complexity
- Unimportant parameters complicate optimization significantly


**Note:** that the parameter grid is defined slightly different, since the Bayeasian search has an internal implementation for `uniform` and `log-uniform` sampling.

In [10]:
# use a full grid over all parameters
param_grid = {
    "average": [True, False],
    "l1_ratio": (0, 1, 'uniform'),
    "alpha": (1e-2, 1e0, 'log-uniform'),
}

# run grid search
bayes_search = BayesSearchCV(clf, param_grid, n_iter=n_iter_search)
start = time()
bayes_search.fit(X, y)

print(
    "BayesSearchCV took %.2f seconds for %d candidate parameter settings."
    % (time() - start, len(bayes_search.cv_results_["params"]))
)
report(bayes_search.cv_results_)

BayesSearchCV took 11.99 seconds for 15 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.993 (std: 0.009)
Parameters: OrderedDict([('alpha', 0.5228830309877981), ('average', False), ('l1_ratio', 0)])

Model with rank: 2
Mean validation score: 0.991 (std: 0.006)
Parameters: OrderedDict([('alpha', 0.031583415023487306), ('average', False), ('l1_ratio', 0)])

Model with rank: 3
Mean validation score: 0.985 (std: 0.007)
Parameters: OrderedDict([('alpha', 0.4421444661338732), ('average', False), ('l1_ratio', 0)])

Model with rank: 3
Mean validation score: 0.985 (std: 0.010)
Parameters: OrderedDict([('alpha', 0.1169672970533936), ('average', False), ('l1_ratio', 0)])

